# Exploratory Data Analysis of Transactions

In [1]:
import pandas as pd

A file has been added to `data/raw` with the name `unclean_transactions.csv`.  This file contains an export of the transactions table from the database.  We'll use this for our exploratory data analysis and transformation steps.

In [2]:
transactions = pd.read_csv('../data/raw/unclean_transactions.csv')

# Display some information about the transactions DataFrame
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10500 non-null  int64 
 1   customer_id       10500 non-null  int64 
 2   transaction_date  10292 non-null  object
 3   amount            10454 non-null  object
dtypes: int64(2), object(2)
memory usage: 328.3+ KB


We can see that there is some missing data in the `transaction_date` and `amount` columns.  We will need to clean this data before we can use it for analysis or further processing.  We can see how many rows we expect to lose as a result cleaning this.

In [3]:
transactions.isnull().any(axis=1).sum()

np.int64(254)

254 rows have missing values in any of the columns.  This is about 0.5% of the total number of rows in the table, so we can safely remove these rows without losing too much data.

In [4]:
# Show how many duplicates there are in the DataFrame
duplicates = transactions.duplicated().sum()
print(f'There are {duplicates} duplicate rows in the transactions DataFrame.')

There are 518 duplicate rows in the transactions DataFrame.


We now know that there are some duplicate rows in the transactions DataFrame.  We will need to remove these duplicates before we can use the data for analysis or further processing.

In [5]:
# Show the different date formats in the transaction_date column
unique_date_formats = transactions['transaction_date'].unique()
print(f'Different date formats in transaction_date: {unique_date_formats[:20]}')  # Show first 20 unique values

Different date formats in transaction_date: ['Mar 05, 2024' '03-01-2024' '16 Oct 2024' 'Mar 06, 2022' '2024-10-19'
 'Apr 17, 2023' '2023-01-31' '23 Jul 2024' '10/04/2023' '15-08-2023'
 '22-07-2023' '2024-04-19' '2022-02-24' 'Nov 12, 2023' '2022-01-20'
 '24 Jan 2022' '04 Oct 2023' 'Jan 10, 2025' '12/10/2023' 'Aug 23, 2023']


This shows that we will need to standardise the date format in the `transaction_date` column.

In [6]:
# Show the different data types in the amount column
amount_types = transactions['amount'].map(type).unique()
print(f'Different data types in amount column: {amount_types}')

Different data types in amount column: [<class 'str'> <class 'float'>]


Now we can see that the amount is either a string or a float.  We will need to convert the amount column to a numeric type before we can use it for analysis or further processing.

---
---

## Cleaning the Data

---

### Epic 2 - Story 3 - Task 2 - Handle Missing Values

We are going to remove any incomplete rows from the DataFrame.  This will remove any rows that have missing values in any of the columns.

In [7]:
transactions = transactions.dropna(subset=["transaction_date"])

# remove rows with null values in amount from the transaction dataframe
transactions = transactions.dropna(subset=["amount"])

# See information about the transactions dataframe
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10246 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10246 non-null  int64 
 1   customer_id       10246 non-null  int64 
 2   transaction_date  10246 non-null  object
 3   amount            10246 non-null  object
dtypes: int64(2), object(2)
memory usage: 400.2+ KB


> We have dropped 254 rows from the DataFrame that had missing values in any of the columns.

In [8]:
# Run a check - this operation will be tested in the pipeline!
transactions.isnull().any(axis=1).sum()

np.int64(0)

---

### Epic 2 - Story 3 - Task 3 - Standardise Date Format

We saw that the `transaction_date` column has a mix of date formats.  We will standardise this to a single format.  You need to examine the date column and then identify and list ALL of the date formats that are present in the column.  You can then use this information to standardise the date format.

In [9]:
# Convert all dates into dd/mm/yyyy format - write function to handle the different types of date formats
def standardise_date(date_str):
    if pd.isna(date_str) or date_str == "":
        return pd.NaT

    formats = [
        "%Y/%m/%d",
        "%Y-%m-%d",
        "%d %b %Y",
        "%b %d, %Y",
        "%d %B %Y",
        "%d-%m-%Y",
        "%d/%m/%Y",
        "%m/%d/%Y",
        "%d/%m/%Y",
    ]
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue

    return pd.NaT


# Apply the parse_date function to the transaction_date column
transactions["transaction_date"] = transactions["transaction_date"].apply(
    standardise_date
)
transactions["transaction_date"] = transactions[
    "transaction_date"
].dt.strftime("%d/%m/%Y")

transactions = transactions.dropna(subset=["transaction_date"])

# Display the DataFrame info
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10246 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10246 non-null  int64 
 1   customer_id       10246 non-null  int64 
 2   transaction_date  10246 non-null  object
 3   amount            10246 non-null  object
dtypes: int64(2), object(2)
memory usage: 400.2+ KB


All dates in the `transaction_date` column are now in the standardised format of `%d/%m/%Y`.

---

### Epic 2 - Story 3 - Task 4 - Convert Amount to Numeric

Find out how many rows can be converted to numeric values in the `amount` column.  This will help us understand how many rows we can use for analysis or further processing.  We specifically want to convert the string to a float and understand how many rows will be converted and how many `NaN` values will be created as a result of this conversion.

In [10]:
# Count convertible vs non-convertible amounts
convertible = (
    pd.to_numeric(transactions["amount"], errors="coerce").notna().sum()
)
print(f"Convertible: {convertible}, NaNs: {len(transactions) - convertible}")

Convertible: 10205, NaNs: 41


The values that generate the `NaN` values are those that cannot be converted to a float.  This includes any non-numeric characters or strings that do not represent a valid number.  We can check what these are.

In [11]:
# Show original values that can't be converted to numeric
unconvertible = transactions[
    pd.to_numeric(transactions["amount"], errors="coerce").isna()
]["amount"].unique()
print(unconvertible)

['INVALID']


This confirms to use that a number of rows in the `amount` column cannot be converted to a float as they contain the string `INVALID`.  We can safely remove these rows from the DataFrame as they will not be useful for analysis or further processing.

In [12]:
# Convert and drop NaNs in one operation
transactions["amount"] = pd.to_numeric(transactions["amount"], errors="coerce")
transactions.dropna(subset=["amount"], inplace=True)

transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10205 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10205 non-null  int64  
 1   customer_id       10205 non-null  int64  
 2   transaction_date  10205 non-null  object 
 3   amount            10205 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 398.6+ KB


We have cleaned the `amount` column by converting it to a numeric type and dropping any rows that contain `NaN` values in this column.  This will allow us to use the `amount` column for analysis or further processing.

This concluldes the cleaning of the `transactions` DataFrame.  We have removed any rows with missing values, standardised the date format, removed duplicates, and converted the `amount` column to a numeric type.  The DataFrame is now ready for analysis or further processing.

---

### Epic 2 - Story 3 - Task 5 - Remove Duplicates

Check to see how many duplicates there are now once the data has been cleaned a little.

In [13]:
# Show how many duplicates there are in the DataFrame
duplicates = transactions.duplicated().sum()
print(f"There are {duplicates} duplicate rows in the transactions DataFrame.")

There are 503 duplicate rows in the transactions DataFrame.


In [14]:
# Drop the duplicates

transactions.drop_duplicates(inplace=True)

transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9702 entries, 0 to 9981
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9702 non-null   int64  
 1   customer_id       9702 non-null   int64  
 2   transaction_date  9702 non-null   object 
 3   amount            9702 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 379.0+ KB


This should have removed the 503 duplicated rows from the DataFrame.

### Reset the indexes

> This was added after the COMPONENT tests for customers data failed due to index conflicts - as we modified the DataFrame, the indexes were no longer sequential.

We can see that the indexes are now out of order, so we will reset them to be sequential again.

The COMPONENT tests for the transactions data set will also need to be updated/added.

In [15]:
transactions.reset_index(drop=True, inplace=True)

---
---

### Epic 2 - Story 3 - Task 6 - Save the Cleaned Data

For testing purposes in the pipeline, it makes sense for us to export the cleaned DataFrame to a CSV file.  This will allow us to use the cleaned data in the pipeline without having to run the cleaning steps again.

In [16]:
transactions.to_csv(
    "../tests/test_data/expected_transactions_clean_results.csv", index=False
)

---

### Epic 2 - Story 3 - Task 7 - Transfer the code from the Jupyter Notebook to a Python script, creating separate functions for each cleaning step

### Epic 2 -Story 3 - Task 8 - Write tests for each cleaning function to ensure they work correctly

### Epic 2 - Story 3 - Task 9 - Create a script to run the cleaning functions in sequence and log the process

### Epic 2 - Story 3 - Task 10 - Add the transaction cleaning script to scripts/run and update any tests accordingly

Jupyter Notebooks do not play nicely with CI/CD pipelines, so we will need to transfer the code from the Jupyter Notebook to a Python script.  We will create separate functions for each cleaning step and then write tests for each function to ensure they work correctly.